In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
# Import modules
import numpy as np # maths

from time import time# timer for debug

    # gather results obtained from the runs
import pickle # open .pkl files where python objects have been saved
from safe import safe # the empty class which is used to save the results
    
    # system commands
import os
    
    # interact with the C++ core and the Python core
import dimers as dim # C++ interface module
import KagomeFunctions as kf # "library" allowing to work on Kagome
import KagomeDrawing as kdraw # "library" allowing to plot Kagome
import Observables as obs #observables that can be used by the run script
import KagomeFT as kft #small functions to compute the FT
import AnalysisFunctions as af # functions to make the analysis code cleaner
    #plots using matplotlib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams.update({'font.size': 14, 'pgf.texsystem':'pdflatex'})

In [ ]:
# Create a folder to save the pictures
foldername = '../../Analysis_PSI_EPFL/Runs_12-07-20_J1J2/'
results_foldername = 'MediumKJ1J2_Corr_L8_J20.02_folder/Results_16-07-20/'
filenamelist = ['MediumKJ1J2_Corr_L8_J20.02_folder/backup']
os.makedirs('./' + foldername + results_foldername, exist_ok = True)

n = len(filenamelist)

In [ ]:
[L, numsites, J1, J2, J3, J3st, J4, nb, num_in_bin, temperatures, nt,
 stat_temps, temperatures_plots, hfields, nh, 
 stat_hfields, hfields_plots, listfunctions, sref, ids2walker] = \
af.LoadParameters(foldername, filenamelist)

s_ijl, ijl_s = kdraw.createspinsitetable(L[0])
print("nb ", nb, "num_in_bin ", num_in_bin)
swapst_th, swapsh_th, swapst, swapsh = af.LoadSwaps(foldername, filenamelist, nb, num_in_bin, nh, nt)

kw = {'binning': False, 'plzplot': True, 'plotmin': 0, 'plotmax': 16}
[t_h_MeanE, t_h_MeanEsq, t_h_varMeanE, t_h_varMeanEsq, C, ErrC] = \
 af.LoadEnergy(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

[t_h_MeanM, t_h_MeanMsq, t_h_varMeanM, t_h_varMeanMsq, Chi, ErrChi] = \
 af.LoadMagnetisation(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

In [ ]:
###########################################
        ## ALGORITHM ANALYSIS ##
###########################################
tidmin = 0
tidmax = nt
af.SwapsAnalysis(L, n, tidmin, tidmax, temperatures, hfields,
                 foldername, results_foldername, swapst, swapsh)

In [ ]:
###########################################
        ## Energy ANALYSIS ##
###########################################
tidmin = 0
tidmax = [len(stat_temps[i]) for i in range(n)]
temperatures_plots = np.array(temperatures_plots)
### Energy
S0 = np.log(2)
kw = {'gscheck': True}
af.BasicPlotsE(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanE, t_h_MeanEsq, t_h_varMeanE,
                t_h_varMeanEsq, C, ErrC, J1, J2, J3, J4, **kw)
plt.show()

In [ ]:
###########################################
        ## MAGNETISATION ANALYSIS ##
###########################################
af.BasicPlotsM(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanM, t_h_MeanMsq, t_h_varMeanM,
                t_h_varMeanMsq, Chi, ErrChi, J1, J2, J3, J4, **kw)
plt.show()

In [ ]:
rmmag = True
kw = {'rmmag':rmmag}

In [ ]:
t_h_MeanFc, t_h_varMeanFc, t_h_MeanSi, t_h_varMeanSi= \
 af.LoadFirstCorrelations(foldername, filenamelist, listfunctions,stat_temps[0:4], stat_hfields, nb, **kw)

In [ ]:
t_h_MeanSs, t_h_varMeanSs, t_h_MeanSi, t_h_varMeanSi, t_h_MeanCorr, t_h_errCorrEstim = \
 af.LoadCentralCorrelations(foldername, filenamelist, listfunctions, sref, stat_temps[0:4], stat_hfields, nb, **kw)
print(t_h_errCorrEstim[0].shape)
print(t_h_MeanFc[0].shape)

In [ ]:
tmin = 0
plt.figure(figsize=(11,9), dpi = 200)
plt.semilogx(temperatures_plots[0][tmin:],t_h_MeanFc[0][tmin:,0,0],'.',label = 'NN')
plt.fill_between(temperatures_plots[0][tmin:],
                t_h_MeanFc[0][tmin:,0,0]-np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]),
                t_h_MeanFc[0][tmin:,0,0]+np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]), alpha = 0.2)
plt.semilogx(temperatures_plots[0][tmin:],t_h_MeanFc[0][tmin:,0,1],'x',label = 'NN2')
plt.fill_between(temperatures_plots[0][tmin:],
                t_h_MeanFc[0][tmin:,0,1]-np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]),
                t_h_MeanFc[0][tmin:,0,1]+np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]), alpha = 0.2)
plt.semilogx(temperatures_plots[0][tmin:],t_h_MeanFc[0][tmin:,0,2],'v',label = 'NN3par')
plt.fill_between(temperatures_plots[0][tmin:],
                t_h_MeanFc[0][tmin:,0,2]-np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]),
                t_h_MeanFc[0][tmin:,0,2]+np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]), alpha = 0.2)
plt.semilogx(temperatures_plots[0][tmin:],t_h_MeanFc[0][tmin:,0,3],'*',label = 'NN3star')
plt.fill_between(temperatures_plots[0][tmin:],
                t_h_MeanFc[0][tmin:,0,3]-np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]),
                t_h_MeanFc[0][tmin:,0,3]+np.sqrt(t_h_errCorrEstim[0][0,tmin:,0,0]), alpha = 0.2)
plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))
plt.xlabel(r"$T/J_1$")
plt.ylabel(r"$\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$")
plt.yticks(np.arange(-0.3,0.7,0.1))
plt.grid(which='both')
plt.legend()
plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations.png")

In [ ]:
tmin=0
tmax = 128

In [ ]:

plt.figure(figsize=(11,9), dpi = 200)
plt.semilogx(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,0],'.',label = 'NN')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,0]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,0]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)

plt.semilogx(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,1],'x',label = 'NN2')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,1]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,1]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)

plt.semilogx(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,2],'v',label = 'NN3par')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,2]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,2]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)

plt.semilogx(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,3],'*',label = 'NN3star')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,3]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,3]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)
plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))
plt.xlabel(r"$T/J_1$")
plt.ylabel(r"$\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$")
plt.grid(which='both')
plt.xlim([0.1, 8])
plt.ylim([-0.35, 0.2])
plt.yticks(np.arange(-0.35,0.25,0.05))
plt.legend()
plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Zoom.png")

In [ ]:
tmin =0
tmax =128
plt.figure(figsize=(11,9), dpi = 200)
plt.plot(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,0],'.',label = 'NN')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,0]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,0]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)
plt.plot(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,1],'x',label = 'NN2')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,1]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,1]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)
plt.plot(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,2],'v',label = 'NN3par')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,2]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,2]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)
plt.plot(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,3],'*',label = 'NN3star')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,3]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,3]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), alpha = 0.2)

plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))
plt.xlabel(r"$T/J_1$")
plt.ylabel(r"$\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$")
plt.xlim([0, 8])
plt.ylim([-0.35, 0.2])
plt.yticks(np.arange(-0.35,0.25,0.05))
plt.grid(which='both')
plt.legend()
plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_ZoomLinear.png")

In [ ]:
tmin = 0
tmax =128
plt.figure(figsize=(11,9), dpi = 200)
plt.plot(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2],'k.',label = 'NN2 - NN3par')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), color = 'k', alpha = 0.2)

plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))
plt.xlabel(r"$T/J_1$")
plt.ylabel(r"$\Delta(\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle)$")
plt.xlim([0, 60])
plt.ylim([-0.2, 0.05])
plt.yticks(np.arange(-0.2,0.075,0.025))
plt.grid(which='both')
plt.legend()
plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference_ZoomLinear.png")

In [ ]:
tmin = 0
tmax =128
plt.figure(figsize=(11,9), dpi = 200)
plt.semilogx(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2],'k.',label = 'NN2 - NN3par')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), color = 'k', alpha = 0.2)

plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))
plt.xlabel(r"$T/J_1$")
plt.ylabel(r"$\Delta(\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle)$")
plt.xlim([0.1, 8])
plt.ylim([-0.2, 0.05])
plt.yticks(np.arange(-0.2,0.075,0.025))
plt.grid(which='both')
plt.legend()
plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference_ZoomLog.png")

In [ ]:
tmin = 0
tmax =128
plt.figure(figsize=(11,9), dpi = 200)
plt.plot(temperatures_plots[0][tmin:tmax],t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2],'k.',label = 'NN2 - NN3par')
plt.fill_between(temperatures_plots[0][tmin:tmax],
                t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2]-np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]),
                t_h_MeanFc[0][tmin:tmax,0,1]-t_h_MeanFc[0][tmin:tmax,0,2]+np.sqrt(t_h_errCorrEstim[0][0,tmin:tmax,0,0]), color = 'k', alpha = 0.2)

plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))
plt.xlabel(r"$T/J_1$")
plt.ylabel(r"$\Delta(\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle)$")
plt.xlim([0.7, 60])
plt.ylim([-0.02, 0.02])
#plt.yticks(np.arange(-0.2,0.075,0.025))
plt.grid(which='both')
plt.legend()
plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference_Measure.png")

In [ ]:
### COMPUTING CORRELATIONS FROM <SISJ> AND <SI> and plotting in 2D
ploth = False
if not ploth:
    tid = 0
    hid = 0
    rid = tid
else:
    tid = 0
    hid = 0
    rid = hid

[corr, errcorr, maxerr] = af.PrepPlot2DCorrelations(rid, n, 
                                                t_h_MeanCorr,
                                                t_h_errCorrEstim,
                                                t_h_MeanSi,
                                                hfields_plots,
                                                temperatures_plots,\
                                                ploth = ploth)
print("t = ",temperatures_plots[0][tid])
print("h = ",hfields_plots[0][hid])

In [ ]:
af.PlotFirstCorrelations(n, L,foldername, results_foldername,  hfields_plots, temperatures_plots,
                         t_h_MeanCorr, t_h_errCorrEstim, sref, \
                         distmax = 2, ploth = ploth, plotFirst = True,
                        t_h_MeanFc = t_h_MeanFc, tmin = tmin, tmax = tmax, **kw)
plt.title(r"$J_2/J_1=$"+"{0}".format(J2[0]))